In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import division
from __future__ import print_function
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# %tensorflow_version 1.14
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy.linalg as la
import scipy.io as sio
import math
import sys
import time
import pdb
import matplotlib.pyplot as plt
import problem, network, train
import blocksparsetoolbox as bst

In [ ]:
np.random.seed(1)
tf.set_random_seed(1)

In [ ]:
# Creating the problem:

In [ ]:
SNR = 30 # signal to noise ratio given in dB
MC = 250 # batch number
prob = problem.block_gaussian(m=50, L=128, B=15, MC=MC, pnz=0.1, SNR_dB=SNR)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,3)
ax[0].imshow(prob.A)
ax[0].set_title('Matrix A')
ax[1].plot(prob.xval[:,0])
ax[1].set_title('x')
ax[2].plot(prob.yval[:,0])
ax[2].set_title('y')


In [ ]:
# creating the network and setup training:

T = 6 # number of layers/iterations

In [ ]:
# computing the analytical weight matrix for ALBISTA and NA-ALBISTA



In [ ]:
layers = network.build_LBISTA(prob, T, initial_lambda=.1)
start = time.time()
training_stages = train.setup_training(layers,prob,trinit=1e-3)
end = time.time()
print( 'Took me {totaltime:.3f} minutes for setup training'.format(totaltime = (end-start)/60))

In [ ]:
# Train!

In [ ]:
sess = train.do_training(training_stages,prob,'trainings/LBISTA_T'+str(T)+'_SNRdB_'+str(SNR)+'batch'+str(MC)+'.npz')

In [ ]:
# Evaluating

In [ ]:
y,x = prob(sess)
MC = x.shape[-1]
t=0
l2norm=np.zeros(((T),MC))
nmse_dbLISTA=np.zeros(((T),MC))
for name, xhat_, var_list in layers:
    if not name=='Linear':
        xhat = sess.run(xhat_, feed_dict={prob.y_: y, prob.x_: x})
        for i in range(0, x.shape[1]):
            nmse_dbLISTA[t,i]=bst.nmse(xhat[:,i, np.newaxis], x[:,i, np.newaxis])
            l2norm[t, i] = bst.l21norm(xhat[:, i]- x[:, i], prob.L, prob.B)
        t+=1

nmse_dbLISTAMean = 10*np.log10(np.mean(np.ma.masked_invalid(nmse_dbLISTA), axis=1))
l2normLISTAMean = np.mean(np.ma.masked_invalid(l2norm), axis=1)
l2normmax = np.max(l2norm, axis=1)

lam = np.zeros(T)
gam = np.zeros(T)
k = 1
for name, xhat_, var_list in layers:
    if not name == 'Linear':
        lam[k-1], gam[k-1] = sess.run([layers[k-1][2][0], layers[k-1][2][1]])
        k = k+1

plt.plot(nmse_dbLISTAMean)